# Problem_Statement :-- 
##### Churn Prediction is one of the most popular big data use cases in business.It consist of detecting customers who are likely to cancel a subscription to a service.
##### Churn is a problem for telecom companies because it is more expensive to acquire a new customer than to keep your existing one from leaving

# OBJECTIVE OF THE PROJECT :-
#### >>To predict customer churn
#### >>Highlighting the main variables/factors influencing customer churn
#### >>Use various ML algorithms to build prediction models,evaluates the accuracy and performance of these models.
#### >>Finding out the best model for our business case & providing executive summary

## Importing libraries and Loading the DATASET

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("telecommunications_churn.csv")

In [ ]:
df.head()

## Data Preprocessing 

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

## Exploratory Data Analysis[EDA]

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class

In [ ]:
%matplotlib inline    
av = AutoViz_Class()
df_av = av.AutoViz("telecommunications_churn.csv")

## Feature Engineering

In [ ]:
#### Feature Engineerin method-1
df.corr()['churn']

In [ ]:
X = df.iloc[:,0:18]
Y = df['churn']

In [ ]:
X

In [ ]:
#### Feature Engineering method-2
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier


In [ ]:
model = DecisionTreeClassifier()

In [ ]:
rfe = RFE(model,n_features_to_select = 6)
fit = rfe.fit(X,Y)

In [ ]:
pd.DataFrame({'Columns':X.columns,"Ranking":fit.ranking_})

In [ ]:
df.corr()['churn']

###### We are comparing with correlation method and RFE method.There is column which have less correlation with thetarget variable.Hencge we are droping the respective columns (evening-calls,night-calls)

In [ ]:
df.drop(['evening_calls','night_calls'],axis =1,inplace= True)

In [ ]:
df.head()

In [ ]:
df.shape

### Feature Selection

In [ ]:
x = df.iloc[:,0:16]
y = df['churn']

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test = train_test_split(x,y,train_size=0.65,random_state=42)

# MODEL BUILDING

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [ ]:
model = []
model.append(('LR',LogisticRegression()))
model.append(('KNN',KNeighborsClassifier()))
model.append(('RF',RandomForestClassifier()))
model.append(('XGBM',XGBClassifier()))
model.append(('SVM',SVC()))
model.append(('DT',DecisionTreeClassifier()))

In [ ]:
results = []
names = []

for name, models in model:
	kfold = KFold(n_splits=10)
	cv_results = cross_val_score(models,x,y,cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

# XGBOOST and RANDOM FOREST has high accuracy when compared to other models

### Hyper Parameter Optimization

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier

kfold = KFold(n_splits=10)
f_model = RandomForestClassifier()

param = {"n_estimators":[50,100,150,200],"criterion":["gini","entropy"]}

grid = GridSearchCV(estimator = f_model,param_grid = param,cv = KFold())

In [ ]:
grid_result = grid.fit(x_train, y_train)

In [ ]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))

In [ ]:
# Building the final model with best parameter
model_final = RandomForestClassifier(criterion= 'gini', n_estimators= 150)
model_final.fit(x_train, y_train)

In [ ]:
# Predicting the model using test data
pred=model_final.predict(x_test)

In [ ]:
#Accuracy for the test data
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

In [ ]:
#confusion metrix
confusion_matrix(pred,y_test)

## Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import set_config
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
steps=[("Standard Scaler",StandardScaler()),
      ("Model",RandomForestClassifier(criterion= 'gini', n_estimators= 150))]

In [ ]:
pipe=Pipeline(steps)

In [ ]:
#for visualizing the pipeline
set_config(display="diagram")
pipe

In [ ]:
X = df.drop("churn",axis=1)
y = df["churn"]

In [ ]:
X

In [ ]:
from sklearn_pandas import DataFrameMapper
X.columns

In [ ]:
pipe.fit(X,y)

## Model Saving

In [ ]:
import pickle
f1 = open("churn_final_data.pkl",mode="wb")

In [ ]:
pickle.dump(pipe,f1)
f1.close()